In [2]:
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from scipy import stats
#import optshrink as opt # package we create
import numpy as np
# import scipy.io as sio
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import seaborn as sns
from shapely.geometry import Point
import geopandas as gp
from geodatasets import get_path
from shapely.geometry import Polygon

In [2]:
clm_cols = ["lon", 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']
atm_cols = ['lon', 'lat', 'year', 'average']
elev_cols = ["lon", 'lat','var' ,'Area', 'elev', 'region']
co2_cols = ['year', 'co2']

In [3]:
dir_path = '/group/moniergrp/TEM_Large_Ensemble/output_files/cru_historical/region_3/'
test_region = pd.read_csv(dir_path+"lat_lon_year_standage_var_hist.csv")

In [4]:
test_region.loc[test_region['variable']==' VEGC '].groupby('forest_type')['monthly_mean'].describe()

,count,mean,std,min,25%,50%,75%,max
forest_type,,,,,,,,
Mixed Temperate Forests,193877.0,9972.807728,6389.850082,0.23,5169.7800,9423.160,14395.7000,32677.83
Temperate Broadleaved Evergreen Forests,3146.0,30.694758,68.310391,0.00,3.7300,4.770,10.0025,448.50
Temperate Coniferous Forests,118545.0,6090.234571,7099.707909,0.00,100.9200,2976.690,10185.7600,34625.83
Temperate Deciduous Forests,73588.0,7921.717661,5568.251295,0.00,3458.9800,6883.925,11440.2700,25683.89
Temperate Forested Wetlands,100666.0,8722.078946,6483.212224,0.34,3850.5425,7258.710,12256.0150,32676.86
Tropical Deciduous Forests,63424.0,6242.732523,5449.997780,0.00,1419.1125,4831.725,10292.3850,21457.13
Tropical Evergreen Forests,45960.0,12844.380645,6117.378927,0.00,9666.6000,13772.215,17245.0975,26214.87
Xeromorphic Forests and Woodlands,154674.0,80.349516,823.360800,0.00,0.0000,0.030,10.4300,224292.81


In [5]:
test_region = test_region.loc[~test_region['current_veg'].isin([6,19])] ###dropping woodlands
test_region = test_region.loc[test_region['monthly_mean'].between(-1e8,1e8)] ###dropping woodlands
test_region.loc[test_region['community_type'].isin([4,5]), 'PFT_group'] = 'Boreal'
test_region.loc[test_region['community_type'].isin([17,20,16,18]), 'PFT_group'] = 'Tropical'
test_region.loc[test_region['community_type'].isin([10,11,25]), 'PFT_group'] = 'Temperate Deciduous'
test_region.loc[test_region['community_type'].isin([9,8]), 'PFT_group'] = 'Temperate Coniferous'
test_region.loc[test_region['community_type'].isin([33]), 'PFT_group'] = 'Temperate Broadleaf'
test_region.loc[test_region['PFT_group'].isna()]

,lon,lat,year,variable,stand_age_interval_min,forest_type,current_veg,community_type,silt_clay,region,...,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PFT_group


In [8]:
test_region

,lon,lat,year,variable,stand_age_interval_min,forest_type,current_veg,community_type,silt_clay,region,...,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PFT_group
0,-91.0,30.5,1700,VEGC,150,Mixed Temperate Forests,8,9,59.45,United_States_of_America,...,16217.46,16285.09,16363.99,16435.51,16491.49,16527.18,16520.96,16459.64,16376.46,Temperate Coniferous
1,-91.0,30.5,1700,VEGC,150,Mixed Temperate Forests,8,10,59.45,United_States_of_America,...,25849.32,26046.32,26131.18,26206.05,26302.12,26324.85,26298.94,26107.16,26024.03,Temperate Deciduous
2,-91.0,30.5,1701,VEGC,150,Mixed Temperate Forests,8,9,59.45,United_States_of_America,...,16229.31,16305.36,16356.64,16341.19,16295.48,16292.66,16288.17,16260.18,16206.63,Temperate Coniferous
3,-91.0,30.5,1701,VEGC,150,Mixed Temperate Forests,8,10,59.45,United_States_of_America,...,25987.98,26177.39,26181.97,26097.25,25985.49,25960.64,25953.64,25930.02,25755.36,Temperate Deciduous
4,-91.0,30.5,1702,VEGC,150,Mixed Temperate Forests,8,9,59.45,United_States_of_America,...,16130.05,16199.18,16292.28,16401.10,16487.05,16540.42,16542.43,16502.02,16439.06,Temperate Coniferous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8336323,-90.0,36.0,2021,NCE,20,Temperate Forested Wetlands,11,11,65.24,United_States_of_America,...,-17.46,32.79,56.81,70.46,64.31,47.91,18.98,-35.68,-41.82,Temperate Deciduous
8336324,-90.0,36.0,2021,NCE,25,Temperate Forested Wetlands,11,11,65.24,United_States_of_America,...,-19.56,38.79,65.85,78.89,69.24,50.25,18.51,-41.01,-48.06,Temperate Deciduous
8336325,-90.0,36.0,2021,NCE,30,Temperate Forested Wetlands,11,11,65.24,United_States_of_America,...,-19.84,37.94,64.67,77.82,68.33,49.28,17.96,-40.94,-47.99,Temperate Deciduous
8336326,-90.0,36.0,2021,NCE,35,Temperate Forested Wetlands,11,11,65.24,United_States_of_America,...,-21.08,44.48,73.01,85.01,71.87,50.72,17.13,-46.24,-54.25,Temperate Deciduous


In [6]:
###all gridded annual input data

grid_monthly = ['/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/tswrf_v11_1700_2021.txt',
'/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/tmp_1700_2021.txt',
'/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/precip_1700_2021.txt',
'/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/dtr_1700_2021.txt',
'/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/vpr_1700_2021.txt',
'/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/wind_1700_2021.txt']

grid_annual = ['/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/Ndep_Trendy.csv',
'/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/Nfer_crop.csv',
'/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/Nfer_pas.csv']

###static gridded data
# grid = [
#     # '/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/cruigbpstxtlf.glb',
# '/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/crutbaselvlf.glb']

grid = '/group/moniergrp/TEM_Large_Ensemble/input_files/cru_regions/region_3/crutbaselvlf.glb'

###co2 data
annual = '/home/smmrrr/TRENDY_Input_Data/atm_chem_data/global_co2_ann_1700_2021.txt'

In [7]:
elev = pd.read_csv(grid, names = elev_cols)
elev

,lon,lat,var,Area,elev,region
0,-119.5,36.0,ELEV,2492,111.6,NAMERICA
1,-119.0,35.5,ELEV,2508,936.2,NAMERICA
2,-119.0,36.0,ELEV,2492,1406.5,NAMERICA
3,-118.5,36.0,ELEV,2492,2400.5,NAMERICA
4,-117.0,34.0,ELEV,2555,1551.6,NAMERICA
...,...,...,...,...,...,...
495,-90.0,34.0,ELEV,2555,99.2,NAMERICA
496,-90.0,34.5,ELEV,2539,108.4,NAMERICA
497,-90.0,35.0,ELEV,2524,96.6,NAMERICA
498,-90.0,35.5,ELEV,2508,81.0,NAMERICA


In [8]:
co2 = pd.read_csv(annual,sep='\s+', names = co2_cols)
co2

,year,co2
0,1700,276.59
1,1701,276.62
2,1702,276.65
3,1703,276.67
4,1704,276.70
...,...,...
318,2018,406.94
319,2019,409.63
320,2020,412.10
321,2021,414.27


In [26]:
i = 0

In [9]:
for i in range(len(grid_annual)):

    file = grid_annual[i]
    base = file.rsplit('/', 1)[-1].rsplit('.', 1)[0]
    print(base)
    data = pd.read_csv(file, names = atm_cols)
    data=data.rename(columns = {'average':base})
    if i == 0:
        atm_chem = data
    else:
        atm_chem = atm_chem.merge(data, on = ['lon', 'lat', 'year'])

Ndep_Trendy
Nfer_crop
Nfer_pas


In [33]:
atm_chem

,lon,lat,year,Ndep_Trendy,Nfer_crop,Nfer_pas
0,-119.5,36.0,1700,46.1921,0.0000,0.0000
1,-119.5,36.0,1701,46.1921,0.0000,0.0000
2,-119.5,36.0,1702,46.1921,0.0000,0.0000
3,-119.5,36.0,1703,46.1921,0.0000,0.0000
4,-119.5,36.0,1704,46.1921,0.0000,0.0000
...,...,...,...,...,...,...
160995,-90.0,36.0,2017,912.8077,6497.5254,2559.7042
160996,-90.0,36.0,2018,927.6445,6697.3172,2551.5778
160997,-90.0,36.0,2019,868.4983,6654.6630,2549.0803
160998,-90.0,36.0,2020,958.4603,6765.6414,2521.2544


In [10]:
for i in range(len(grid_monthly)):

    file = grid_monthly[i]
    base = file.rsplit('/', 1)[-1].rsplit('_1700', 1)[0]
    print(base)
    data = pd.read_csv(file, names = clm_cols)[['lon', 'lat', 'year','average']]
    data=data.rename(columns = {'average':base})
    # print(data.dtypes)
    if i == 0:
        climate = data
    else:
        climate = climate.merge(data, on = ['lon', 'lat', 'year'])

tswrf_v11
tmp
precip
dtr
vpr
wind


In [11]:
input_data = climate.merge(atm_chem, on = ['lon','lat','year']).merge(
elev, on = ['lon','lat']
).merge(co2, on = 'year')

In [12]:
test_region.columns

Index(['lon', 'lat', 'year', 'variable', 'stand_age_interval_min',
       'forest_type', 'current_veg', 'community_type', 'silt_clay', 'region',
       'file', 'land_area', 'cohort_area', 'monthly_mean_weight', 'Jan_weight',
       'Feb_weight', 'Mar_weight', 'Apr_weight', 'May_weight', 'Jun_weight',
       'Jul_weight', 'Aug_weight', 'Sep_weight', 'Oct_weight', 'Nov_weight',
       'Dec_weight', 'monthly_mean', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'PFT_group'],
      dtype='object')

In [13]:
model_data = test_region.loc[(test_region['variable']==' VEGC ') & (test_region['PFT_group'] == 'Temperate Coniferous')
               ,['lon', 'lat', 'year','stand_age_interval_min','silt_clay','monthly_mean']].merge(
 input_data, on = ['lon','lat','year']
)

In [14]:
model_data.columns

Index(['lon', 'lat', 'year', 'stand_age_interval_min', 'silt_clay',
       'monthly_mean', 'tswrf_v11', 'tmp', 'precip', 'dtr', 'vpr', 'wind',
       'Ndep_Trendy', 'Nfer_crop', 'Nfer_pas', 'var', 'Area', 'elev', 'region',
       'co2'],
      dtype='object')

In [16]:
model_data.to_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/region_3_test_model_data_conif_vegc.csv', index=False)

In [3]:
model_data = pd.read_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/region_3_test_model_data_conif_vegc.csv')

In [10]:
unique_bins = model_data.stand_age_interval_min.unique()
bin_to_ordinal = {bin_val: idx for idx, bin_val in enumerate(unique_bins)}
print(bin_to_ordinal)
model_data['ordinal_stand_age'] = model_data['stand_age_interval_min'].map(bin_to_ordinal)
model_data['ordinal_stand_age'].describe()

{150: 0, 0: 1, 5: 2, 10: 3, 15: 4, 20: 5, 25: 6, 30: 7, 35: 8, 40: 9, 45: 10, 50: 11, 55: 12, 60: 13, 65: 14, 70: 15, 75: 16, 80: 17, 85: 18, 90: 19, 95: 20, 100: 21, 125: 22}


count    287630.000000
mean          5.893586
std           5.567873
min           0.000000
25%           1.000000
50%           5.000000
75%           9.000000
max          22.000000
Name: ordinal_stand_age, dtype: float64

In [ ]:
# X = np.array(model_data.loc[model_data['year']>=2021,['tswrf_v11', 'tmp', 'precip', 'dtr', 'vpr', 'wind','stand_age_interval_min']])
X = np.array(model_data.loc[model_data['year']>=2000,['ordinal_stand_age', 'tmp']])
y = np.array(model_data.loc[model_data['year']>=2000,['monthly_mean']])

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

kernel = DotProduct() + WhiteKernel()

gpr = GaussianProcessRegressor(kernel=None,
      random_state=0).fit(X, y)

gpr.score(X, y)